<a href="https://colab.research.google.com/github/Ramane23/peft_fine_tuning/blob/main/peft_fine_tunning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
## Step 1: Install Required Libraries
!pip install transformers datasets evaluate peft torch

  Using cached datasets-3.4.0-py3-none-any.whl.metadata (19 kB)
  Using cached evaluate-0.4.3-py3-none-any.whl.metadata (9.2 kB)
  Using cached dill-0.3.8-py3-none-any.whl.metadata (10 kB)
  Using cached xxhash-3.5.0-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (12 kB)
  Using cached multiprocess-0.70.16-py311-none-any.whl.metadata (7.2 kB)
  Using cached nvidia_cuda_nvrtc_cu12-12.4.127-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.4.127-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.4.127-py3-none-manylinux2014_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-9.1.0.70-py3-none-manylinux2014_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.4.5.8-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.2.1.3-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.5.147-py3-non

In [2]:
## Step 2: Import Basic Libraries
import numpy as np
import pandas as pd
import torch

In [3]:
## Step 3: Import HuggingFace Libraries
from datasets import load_dataset
import evaluate

# Import libraries for working with pre-trained models
from transformers import (
    AutoModelForSequenceClassification,  # For loading models for classification
    AutoTokenizer,                       # For tokenizing text input
    TrainingArguments,                   # For configuring the training process
    Trainer                              # For handling the training loop
)

# Import PEFT-specific libraries
from peft import (
    LoraConfig,                          # For configuring LoRA
    get_peft_model,                      # For applying PEFT to a model
    TaskType,                            # For specifying the task type
    PeftModel                            # For loading saved PEFT models
)

In [4]:
## Step 4: Load a Dataset
# Load the emotion dataset
# This dataset contains text samples labeled with emotions
dataset = load_dataset("emotion")


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/9.05k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/1.03M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/127k [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/129k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/16000 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/2000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/2000 [00:00<?, ? examples/s]

In [5]:
# Display basic information about the dataset
print(dataset)

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 16000
    })
    validation: Dataset({
        features: ['text', 'label'],
        num_rows: 2000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 2000
    })
})


In [6]:
# Show the first few examples
print("\n--- Examples from the dataset ---")
for i in range(3):
    print(f"Example {i+1}:")
    print(f"Text: {dataset['train'][i]['text']}")
    print(f"Label: {dataset['train'][i]['label']}")
    print()


--- Examples from the dataset ---
Example 1:
Text: i didnt feel humiliated
Label: 0

Example 2:
Text: i can go from feeling so hopeless to so damned hopeful just from being around someone who cares and is awake
Label: 0

Example 3:
Text: im grabbing a minute to post i feel greedy wrong
Label: 3



In [7]:
## Step 5: Explore the Dataset
# Get information about the labels
label_names = dataset['train'].features['label'].names
num_labels = len(label_names)

# Create mappings between label IDs and names
id2label = {i: label for i, label in enumerate(label_names)}
label2id = {label: i for i, label in enumerate(label_names)}

print(f"the list of labels is {label_names}")
print(f"Number of labels: {num_labels}")
print(f"Labels: {id2label}")

the list of labels is ['sadness', 'joy', 'love', 'anger', 'fear', 'surprise']
Number of labels: 6
Labels: {0: 'sadness', 1: 'joy', 2: 'love', 3: 'anger', 4: 'fear', 5: 'surprise'}


In [8]:
# Get dataset sizes
print(f"\nTrain set size: {len(dataset['train'])}")
print(f"Validation set size: {len(dataset['validation'])}")
print(f"Test set size: {len(dataset['test'])}")


Train set size: 16000
Validation set size: 2000
Test set size: 2000


In [9]:
## Step 6: Load a Pre-trained Model and Tokenizer
# Load the pre-trained GPT-2 model configured for sequence classification
model = AutoModelForSequenceClassification.from_pretrained(
    "gpt2",
    num_labels=num_labels,        # Number of classes for classification
    id2label=id2label,            # Mapping from ID to label name
    label2id=label2id,            # Mapping from label name to ID
)

# Load the GPT-2 tokenizer
# The tokenizer converts text to token IDs that the model can understand
tokenizer = AutoTokenizer.from_pretrained("gpt2")

# GPT-2 doesn't have a padding token by default, so we set it to the EOS token
tokenizer.pad_token = tokenizer.eos_token

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at gpt2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [10]:
# Tell the model to use our defined padding token
model.config.pad_token_id = tokenizer.pad_token_id

In [11]:
# Display model information
print(f"Model loaded: {model.__class__.__name__}")
print(f"Model size: {model.num_parameters():,} parameters")

Model loaded: GPT2ForSequenceClassification
Model size: 124,444,416 parameters


In [12]:
## Step 7: Define the Preprocessing Function
# Define a function to tokenize and prepare data for the model
def preprocess_function(examples):
    """
    Tokenize the input texts and prepare them for the model.

    Args:
        examples: A batch of examples from the dataset

    Returns:
        A dictionary of tokenized inputs
    """
    # Tokenize the texts
    return tokenizer(
        examples["text"],              # The text to tokenize
        truncation=True,               # Truncate texts longer than max_length
        padding="max_length",          # Pad all sequences to max_length
        max_length=128,                # Maximum sequence length
        return_tensors="pt"            # Return PyTorch tensors
    )

In [13]:
# Apply preprocessing to the entire dataset
# The batched=True option makes this process faster
tokenized_dataset = dataset.map(
    preprocess_function,
    batched=True,
    remove_columns=["text"]  # Remove the original text as it's no longer needed
)

print("Dataset tokenized successfully!")

# Verify the dataset structure
print(f"\nFeatures in the processed dataset: {list(tokenized_dataset['train'].features.keys())}")

Map:   0%|          | 0/16000 [00:00<?, ? examples/s]

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

Dataset tokenized successfully!

Features in the processed dataset: ['label', 'input_ids', 'attention_mask']


In [14]:
## Step 9: Set Up Evaluation Metrics
# Load the accuracy metric for evaluation
accuracy_metric = evaluate.load("accuracy")

# Define a function to compute metrics from model predictions
def compute_metrics(eval_pred):
    """
    Compute evaluation metrics from predictions.

    Args:
        eval_pred: EvalPrediction object containing predictions and labels

    Returns:
        Dictionary of metrics
    """
    # Extract predictions and labels
    predictions, labels = eval_pred

    # Get the predicted class (highest logit)
    predictions = np.argmax(predictions, axis=1)

    # Calculate and return accuracy
    return accuracy_metric.compute(predictions=predictions, references=labels)

In [15]:
## Step 10: Evaluate the Base Model
## let's evaluate the base model before fine-tuning
# Set up training arguments for evaluation only
base_eval_args = TrainingArguments(
    output_dir="./results/base_eval",  # Directory to save results
    per_device_eval_batch_size=8,      # Batch size for evaluation
    do_train=False,                    # No training
    do_eval=True,                      # Only evaluation
    report_to="none"                   # to disable wandb
)


# Create a trainer for evaluation
base_trainer = Trainer(
    model=model,                       # The base model
    args=base_eval_args,               # Training arguments
    tokenizer=tokenizer,               # Tokenizer
    compute_metrics=compute_metrics,   # Metrics function
    eval_dataset=tokenized_dataset["validation"]  # Validation dataset
)

<ipython-input-15-3dfd41d770dd>:14: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  base_trainer = Trainer(


In [22]:
# Evaluate the base model
print("Evaluating base model...")
base_eval_results = base_trainer.evaluate()
print(f"Base model accuracy: {base_eval_results['eval_accuracy']:.4f}")

Evaluating base model...


Base model accuracy: 0.2750


In [16]:
## Step 11: Set Up PEFT with LoRA
# Make sure this import is executed before using LoraConfig
from peft import LoraConfig, get_peft_model, TaskType, PeftModel

# Define LoRA configuration
# LoRA adds low-rank adaptation matrices to specific weights in the model
peft_config = LoraConfig(
    task_type=TaskType.SEQ_CLS,          # Sequence classification task
    r=8,                                 # Rank of the update matrices
    lora_alpha=32,                       # Alpha parameter for LoRA scaling
    lora_dropout=0.1,                    # Dropout probability for LoRA layers
    target_modules=["c_attn", "c_proj"], # Attention modules to apply LoRA to
    bias="none",                         # Don't train bias parameters
)

# Create a PEFT model by applying LoRA to the base model
peft_model = get_peft_model(model, peft_config)

/usr/local/lib/python3.11/dist-packages/peft/tuners/lora/layer.py:1264: UserWarning: fan_in_fan_out is set to False but the target module is `Conv1D`. Setting fan_in_fan_out to True.
  warnings.warn(


In [17]:
## Step 12: Examine Trainable Parameters
# Print information about trainable parameters
print("--- Trainable Parameters Information ---")
print(peft_model.print_trainable_parameters())

--- Trainable Parameters Information ---
trainable params: 815,616 || all params: 125,260,032 || trainable%: 0.6511
None


In [18]:
#Calculate percentage of trainable parameters
total_params = model.num_parameters()
trainable_params = sum(p.numel() for p in peft_model.parameters() if p.requires_grad)
print(f"Total parameters: {total_params:,}")
print(f"Trainable parameters: {trainable_params:,}")
print(f"Percentage of trainable parameters: {trainable_params/total_params*100:.4f}%")

Total parameters: 125,260,032
Trainable parameters: 815,616
Percentage of trainable parameters: 0.6511%


In [19]:
## Step 13: Set Up Training Arguments
##Configure the training process
# Set up training arguments
training_args = TrainingArguments(
    output_dir="./results/peft_model",    # Directory to save results and checkpoints
    learning_rate=5e-4,                   # Learning rate
    per_device_train_batch_size=8,        # Batch size for training
    per_device_eval_batch_size=8,         # Batch size for evaluation
    num_train_epochs=1,                   # Number of training epochs
    weight_decay=0.01,                    # Weight decay for regularization
    evaluation_strategy="epoch",          # When to evaluate (each epoch)
    save_strategy="epoch",                # When to save checkpoints (each epoch)
    load_best_model_at_end=True,          # Load the best model at the end of training
    push_to_hub=False,                    # Don't push to the Hugging Face Hub
    report_to="none"                      # to disable wandb
)

## Step 14: Create a Trainer for Fine-tuning
## Set up the trainer with our PEFT model
# Create a trainer for fine-tuning
trainer = Trainer(
    model=peft_model,                       # The PEFT model
    args=training_args,                     # Training arguments
    train_dataset=tokenized_dataset["train"],  # Training dataset
    eval_dataset=tokenized_dataset["validation"],  # Validation dataset
    tokenizer=tokenizer,                    # Tokenizer
    compute_metrics=compute_metrics,        # Metrics function
)

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
<ipython-input-19-fda74461081b>:21: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [20]:
## Step 15: Train the PEFT Model
# Train the model
print("Training PEFT model...")
train_results = trainer.train()

# Print training results
print(f"Training loss: {train_results.training_loss:.4f}")

Training PEFT model...


Epoch,Training Loss,Validation Loss


Epoch,Training Loss,Validation Loss,Accuracy
1,0.253700,0.219581,0.926500


Training loss: 0.5317


In [24]:
## Step 16: Evaluate the Fine-tuned Model
# Evaluate the PEFT model after training
print("Evaluating PEFT model...")
#peft_eval_results = trainer.evaluate()
#print(f"PEFT model accuracy: {peft_eval_results['eval_accuracy']:.4f}")

Evaluating PEFT model...


In [25]:
## Step 17: Save the Fine-tuned Model
# Save the PEFT model
peft_model.save_pretrained("./peft_model_saved")
print("PEFT model saved to ./peft_model_saved")

PEFT model saved to ./peft_model_saved


In [26]:
## Step 18: Load the Saved Model
# Load the saved PEFT model
print("Loading saved PEFT model...")
loaded_peft_model = PeftModel.from_pretrained(
    model,                     # The base model
    "./peft_model_saved",      # Path to saved PEFT model
    is_trainable=False         # Set to False for inference
)

Loading saved PEFT model...


In [27]:
## Step 19: Evaluate the Loaded Model on Test Set
# Set up evaluation arguments
loaded_eval_args = TrainingArguments(
    output_dir="./results/loaded_eval",  # Directory to save results
    per_device_eval_batch_size=8,        # Batch size for evaluation
    do_train=False,                      # No training
    do_eval=True,                        # Only evaluation
    report_to="none"                     # to disable wandb
)

# Create a trainer for evaluation
loaded_trainer = Trainer(
    model=loaded_peft_model,             # The loaded PEFT model
    args=loaded_eval_args,               # Training arguments
    tokenizer=tokenizer,                 # Tokenizer
    compute_metrics=compute_metrics,     # Metrics function
    eval_dataset=tokenized_dataset["test"]  # Test dataset
)

# Evaluate the loaded model on the test set
print("Evaluating loaded PEFT model on test set...")
#loaded_eval_results = loaded_trainer.evaluate()
#print(f"Loaded PEFT model accuracy on test set: {loaded_eval_results['eval_accuracy']:.4f}")

Evaluating loaded PEFT model on test set...


<ipython-input-27-200a77d92ff3>:12: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  loaded_trainer = Trainer(


In [28]:
## Step 20: Evaluate Base Model on Test Set
# Evaluate the base model on the test set for fair comparison
base_trainer_test = Trainer(
    model=model,
    args=base_eval_args,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
    eval_dataset=tokenized_dataset["test"]
)

print("Evaluating base model on test set...")
base_test_results = base_trainer_test.evaluate()
print(f"Base model accuracy on test set: {base_test_results['eval_accuracy']:.4f}")

<ipython-input-28-23a166b00bcd>:3: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  base_trainer_test = Trainer(


Evaluating base model on test set...


Base model accuracy on test set: 0.1125


In [32]:
## Step 22: Function for Making Predictions
def predict_emotion(text):
    """
    Predict the emotion of a text using the fine-tuned model.

    Args:
        text: The input text

    Returns:
        The predicted emotion label and probability
    """
    # Tokenize the input
    inputs = tokenizer(
        text,                      # The raw text to analyze
        return_tensors="pt",       # Return PyTorch tensors (as opposed to NumPy arrays or TensorFlow tensors)
        padding=True,              # Add padding to make uniform length (needed for batched inference)
        truncation=True,           # Cut off text that's too long for the model
        max_length=128             # Maximum token length to process
    )

    # Get predictions from the model
    with torch.no_grad():          # Disable gradient calculation for inference (saves memory and is faster)
        outputs = loaded_peft_model(**inputs)  # Pass all tokenizer outputs as keyword arguments to the model
                                             # The ** unpacks the dictionary from tokenizer into separate arguments
                                             # e.g., input_ids=inputs["input_ids"], attention_mask=inputs["attention_mask"]

    # Get the predicted class and probability
    probabilities = torch.nn.functional.softmax(outputs.logits, dim=-1)  # Convert logits to probabilities using softmax
                                                                        # dim=-1 means apply softmax across the last dimension (classes)

    predicted_class = torch.argmax(probabilities, dim=-1).item()  # Find the class with highest probability
                                                                 # argmax returns the index, item() converts tensor to Python scalar

    predicted_prob = probabilities[0, predicted_class].item()    # Extract the probability value for the predicted class
                                                                # [0, predicted_class] accesses first batch item and the prediction index

    return id2label[predicted_class], predicted_prob  # Return both the emotion label (mapped from the numeric index)
                                                     # and the confidence score (probability)

In [33]:
## Step 23: Make Predictions
# Test texts for prediction
test_texts = [
    "I'm feeling really happy today!",
    "That movie made me so sad, I cried.",
    "I'm furious about what happened.",
    "I'm not sure how to feel about this.",
    "I just got a promotion, I'm ecstatic!"
]

# Make predictions
print("===== Example Predictions =====")
for text in test_texts:
    emotion, probability = predict_emotion(text)
    print(f"Text: {text}")
    print(f"Predicted emotion: {emotion} (probability: {probability:.4f})")
    print()

===== Example Predictions =====
Text: I'm feeling really happy today!
Predicted emotion: fear (probability: 0.9440)

Text: That movie made me so sad, I cried.
Predicted emotion: fear (probability: 0.9334)

Text: I'm furious about what happened.
Predicted emotion: fear (probability: 0.9516)

Text: I'm not sure how to feel about this.
Predicted emotion: fear (probability: 0.9499)

Text: I just got a promotion, I'm ecstatic!
Predicted emotion: fear (probability: 0.9464)

